# AutoEncoder 실습 : 신용카드 이상거래 감지

## A. 데이터 살펴보기

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
df = pd.read_csv('data/creditcard.csv')
df = df.drop('Time',axis=1)

In [0]:
X = df.drop('Class',axis=1).values
y = df['Class'].values

## B. 자료의  Min-max 표준화


In [0]:
X -= X.min(axis=0)
X /= X.max(axis=0)

## C. 데이터 분리하기

In [0]:
input_train, input_test, label_train, label_test = train_test_split(X, y, test_size = 0.2)

## D. 오토인코더 신경망 모델 수립

### 신경망 구조 정의

In [0]:
from keras.models import Model
from keras.layers import Input, Dense

input_dim = input_train.shape[1]

input_data = Input(shape = (input_dim,))
encoder = Dense(12, activation='tanh')(input_data)
decoder = Dense(29, activation='sigmoid')(encoder)
autoencoder = Model(input_data, decoder)

### 옵티마이저 선택 및 모델 컴파일

In [0]:
autoencoder.compile(optimizer = 'adam', 
                    loss = 'mean_squared_error',
                   metrics = ['accuracy'])

### 모델 Fitting

In [0]:
autoencoder.fit(input_train, 
                input_train, 
                epochs = 20, 
                batch_size = 128,
                validation_data = (input_test, input_test)).history

In [0]:
pred = autoencoder.predict(input_test)

## E. Label 예측하기

In [0]:
mse = np.mean(np.power(input_test - pred, 2), axis=1)
error_df = pd.DataFrame({'reconstruction_error': mse,
                        'true_class': label_test})

In [0]:
error_df.describe()

In [0]:
threshold = 0.003


groups = error_df.groupby('true_class')
fig, ax = plt.subplots()

for name, group in groups:
    ax.plot(group.index, 
            group.reconstruction_error, 
            marker='o', 
            ms=3.5, 
            linestyle='',
            label= "Fraud" if name == 1 else "Normal")
ax.hlines(threshold, 
          ax.get_xlim()[0], 
          ax.get_xlim()[1], 
          colors="r", 
          zorder=100, 
          label='Threshold')
ax.legend()
plt.title("Reconstruction error for different classes")
plt.ylabel("Reconstruction error")
plt.xlabel("Data point index")
plt.show();

In [0]:
import seaborn as sns

label_pred = [1 if e > threshold else 0 for e in error_df.reconstruction_error.values]
conf_matrix = confusion_matrix(error_df.true_class, label_pred)

plt.figure(figsize=(12, 12))
sns.heatmap(conf_matrix, 
            xticklabels = ["Normal", "Fraud"], 
            yticklabels = ["Normal", "Fraud"], 
            annot=True, 
            fmt="d");
plt.title("Confusion matrix")
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.show()

## F. 결과 시각화

### 실제 값과 예측값 결과

In [0]:
import matplotlib.pyplot as plt

pred_value = pred[9]
true_value = input_test[9]
idx = np.arange(len(pred_value))

def GetPlot(pred_value, true_value, idx, width=0.7):
    
    fig = plt.figure(figsize=(10,7))
    plt.bar(idx, 
            pred_value, 
            width=width,
            color='b', 
            label='Predicted Value')

    plt.bar([i+0.25*width for i in idx], 
            true_value,
            width = 0.5*width, 
            color = 'r', 
            alpha = 0.5, 
            label = 'True Value')

    plt.xticks(idx + width/2.,
           ['V{}'.format(i) for i in range(len(pred_value))] )

    plt.legend()
    plt.show()
    
GetPlot(pred_value, true_value, idx)

### Encoder 모델의 시각화 with T-SNE

오토 인코더의 장점은 고차원으로 구성된 이상거래를 저차원으로 표현하여 이상거래의 패턴을 포착할 수 있다는 점이다.

고차원 데이터의 2차원 표현에 많이 쓰이는 `T-SNE(T-Stochastic Neighbor Embedding)`을 이용해 인코더 모델을 시각화해보자.

인코더 모델을 만드려면 `input_data`가 입력되어 인코딩 된 상태(Encoded layer)에서 mapping 되도록 Encoder 단계까지의 모델을 다시 정의해야 한다.

In [0]:
from sklearn.manifold import TSNE

In [0]:
encoder = Model(input_data, encoded_layer)
enc_pred = encoder.predict(input_test)
tsne = TSNE(verbose=1, n_iter=1000)

res = tsne.fit_transform(enc_pred)
fig = plt.figure()

scatter = plt.scatter(res[:,0], 
                      res[:,1], 
                      c = label_test, 
                      cmap = 'coolwarm', 
                      s = 
                      0.6)

scatter.axes.get_xaxis().set_visible(False)
scatter.axes.get_yaxis().set_visible(False)

위 시각화 결과를 살펴보면, 거래 패턴이 상호 구분되는 몇 가지 뚜렷한 `Cluster`들을 발견할 수 있다. 

또한, 이들 중에는 정상 거래와 명백히 구분되는 이상 거래의 `Cluster`를 발견할 수 있다. 

이처럼 비지도 학습 기법의 일환인 `AutoEncoder`를 이용하면 주어진 Label을 분류할 뿐 아니라 거래의 여러가지 패턴을 파악해볼 수 있다.